## 해설

#### 1. 불러오기

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
train_x = train.drop(columns=['HR'])
train_y = train['HR']

test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

#### 2. EDA

데이터분석을 위해 가장 먼저 해야할 일은 바로 데이터를 살펴보는 것입니다. <br> 주어진 데이터를 살펴보며 데이터의 모양과 대락적인 내용을 파악하고, 어떤 분석이 가능한지 확인하여 방향을 잡는 과정이 필요합니다. 

#### 2-1. 데이터 자료형과 통계량 확인

info(), describe() 메소드를 활용하여 데이터의 자료형(Dtype)과 이상치나 결측치가 존재하는지 확인합니다.

In [ ]:
# 각 변수들의 데이터 타입을 확인합니다.
train_x.info()

In [ ]:
# 각 변수들의 데이터 타입을 확인합니다.
train.describe()

#### 2-2. 데이터 시각화

변수들간 상관관계를 보여주는 pairplot, 연속형 변수의 분포를 보여주는 distplot, 그리고 범주형 변수의 빈도를 보여주는 countplot을 이용해서 EDA를 진행합니다.

In [ ]:
plt.figure(figsize=(15,15))
sns.pairplot(train)
plt.show()

#E_Status가 범부형 변수임을 확인

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(20,20))

sns.distplot(x=train_x['CT'], ax=axes[0][0]).set_title('CT')
sns.distplot(x=train_x['MS'], ax=axes[0][1]).set_title('MS')
sns.distplot(x=train_x['MC'], ax=axes[0][2]).set_title('MC')
sns.distplot(x=train_x['B_Depth'], ax=axes[1][0]).set_title('B_Depth')
sns.distplot(x=train_x['LAT_ACCEL'], ax=axes[1][1]).set_title('LAT_ACCEL')
sns.distplot(x=train_x['LONG_ACCEL'], ax=axes[1][2]).set_title('LONG_ACCEL')
sns.distplot(x=train_x['E_Status'], ax=axes[2][0]).set_title('E_Status')
sns.distplot(x=train_x['A_Depth'], ax=axes[2][1]).set_title('A_Depth')
sns.distplot(x=train_x['F_Economy'], ax=axes[2][2]).set_title('F_Economy')
sns.distplot(x=train_x['Inhibit'], ax=axes[3][0]).set_title('Inhibit')
sns.distplot(x=train_x['Elevation'], ax=axes[3][1]).set_title('Elevation')
sns.distplot(x=train_x['HR'], ax=axes[3][2]).set_title('HR')

plt.show()

#['CT', 'MS', 'MC', 'LAT_ACCEL', 'LONG_ACCEL', 'Elevation']의 경우, 분포가 중앙으로 몰려있는 형태
#['B_Depth', 'A_Depth', 'F_Economy', 'HR']의 경우, 분포가 한 쪽으로 치우친 형태
#['E_Status', 'Inhibit']에 불균형 존재

#### 2-3. 다중공선성과 상관관계 확인

하나의 독립변수가 다른 여러 개의 독립변수들로 잘 예측되는 다중공선성을 확인하기 위해, 분산팽창계수(VIF)를 구하고, 히트맵을 사용하여 각 변수 간의 상관계수를 시각적으로 확인하겠습니다. <br>
다중공선성이 있는 경우, 상관계수가 높은 변수 중 하나를 제거하거나 주성분 분석(PCA)과 같은 차원 축소 기법을 사용하여 변수를 줄입니다.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
feature = train_x.drop(columns=['Inhibit'])

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(feature.values, i) for i in range(feature.shape[1])]
vif['features'] = feature.columns
picked = list(vif[vif['VIF Factor'] > 10].features)
print(picked)

vif

In [ ]:
temp = train.corr()
#그림 사이즈 지정
fig, ax = plt.subplots(figsize=(20,20))

#삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래쪽 삼각형에 False)
mask = np.zeros_like(temp)
mask[np.triu_indices_from(mask)] = True

#히트맵 그리기
sns.heatmap(temp,
            cmap='RdYlBu_r',
            annot=True,
            mask=mask,
            linewidths = .5,
            cbar_kws={'shrink': .5},
            vmin = -1, vmax = 1)

plt.show()

#### 2-4. 인사이트 : 변수선택

엄밀한 기준은 없으나 보통 분산팽창계수(VIF)가 10보다 크면 다중공선성이 있다고 판단합니다. (5를 기준으로 하기도 함.) <br>
'LONG_ACCEL'과 'A_Depth'변수는 분산팽창계수가 5이상이며, 히트맵에서 높은 상관관계를 보입니다. <br>
이번 분석에서는 'LONG_ACCEL'변수를 사용하지 않겠습니다. <br>
drop() 메소드의 columns 인자에 제거할 변수를 리스트에 담아 넣어줍니다.

In [ ]:
train_x = train_x.drop(columns=['LONG_ACCEL'])
test_x = test_x.drop(columns=['LONG_ACCEL'])

#### 3. 데이터전처리

컴퓨터는 숫자만 계산 가능하기 때문에, 문자형으로 이루어진 범주형 변수들을 분석에 사용하려면 숫자형으로 변환해주어야 합니다. <br>
[주의] 학습(fit)에는 반드시 학습데이터만 사용해야 합니다.

In [ ]:
from sklearn.preprocessing import LabelEncoder

#데이터 타입이 object인 Inhibit column을 수치화합니다.
qual_col = ['Inhibit']
for i in qual_col:
    le = LabelEncoder()
    #학습데이터의 Inhibit column에 존재하는 값으로 Label Encoder를 fit
    le = le.fit(train_x[i])
    #그 후 학습데이터로 fit된 Label Encoder를 사용해서 학습데이터의 Inhibit 컬럼을 수치화
    train_x[i] = le.transform(train_x[i])

    #학습데이터에는 없고, 테스트데이터에만 존재하는 Label이 있을 수 있으므로 for문으로 예외처리
    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    #[중요] 단, 테스트데이터롤 Label Encoder를 fit한 후 transform을 수행하는 것은 Data Leakage에 해당하므로 주의 -> 무조건 학습데이터로 fit
    train_x[i] = le.transform(test_x[i])


    print('수치화 완료')

#### 3-1. One-Hot Encoding

Label Encoding을 사용하면 범주형 변수 간에 순서가 없는 경우에도 변수 값 사이에 순서가 생길 수 있습니다. <br>
예를 들어, '빨강', '파랑', '노랑'을 각각 1,2,3으로 변환한 경우, 머신러닝 알고리즘이 이를 순서가 있는 값으로 인식할 수 있습니다. <br>
이는 올바르지 않은 정보를 제공할 수 있으며, 머신러닝 모델의 성능을 저하시킬 수 있습니다. <br>
<br>
반면, One-Hot Encoding은 각 카테고리에 대해 새로운 이진 변수를 생성하고, 해당 변수의 값이 1 또는 0이 되도록 변환합니다. <br>
이 방법은 각 카테고리 사이에 순서가 없음을 보장하며, 머신러닝 알고리즘이 범주형 변수를 적절히 처리하도록 도와줍니다. <br>
<br>
get_dummies 메소드를 활용하여 원핫인코딩을 적용하겠습니다.

In [ ]:
train_x = pd.get_dummies(train_x, columns = ['Inhibit', 'E_Status'])
test_x = pd.get_dummies(test_x, columns=['Inhibits', 'E_Status'])
# [중요] 단, 학습데이터와 테스트데이터의 카테고리 수가 일치하지 않을 때는 생성되는 변수의 수가 달라지는 것에 주의합니다.

In [ ]:
train_x.columns

In [ ]:
test_x.columns

#### 3-2. 스케일링

스케일링을 통해 변수들의 데이터 분포나 범위를 조정하여 변수마다 데이터 값의 범위 차이가 클 경우 발생되는 자료의 오버플로우(overflow)나 언더플로우(underflow)를 방지할 수 있습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#각 변수들의 분포를 고려하여 스케일링을 진행합니다.
#분포가 중앙으로 몰려있는 경우, 최솟값을 0, 최댓값을 1로 변환하는 MinMaxScaler 적용
#분포가 한 쪽으로 치우친 형태의 경우, 평균과 표준편차를 이용한 StandardScaler 적용
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

scaled_train_x = train_x.copy()
scaled_train_x[['B_Depth', 'A_Depth', 'F_Economy']] = minmax_scaler.fit_transform(train_x[['B_Depth', 'A_Depth', 'F_Economy']])
scaled_train_x[['CT', 'MS', 'MC', 'LAT_ACCEL', 'Elevation']] = standard_scaler.fit_transform(train_x[['CT','MS','MC','LAT_ACCEL', 'Elevation']])

scaled_test_x = test_x.copy()
# [주의] 단, 테스트데이터에는 transform만 적용
scaled_test_x[['B_Depth', 'A_Depth', 'F_Economy']] = minmax_scaler.transform(test_x[['B_Depth', 'A_Depth', 'F_Economy']])
scaled_test_x[['CT', 'MS', 'MC', 'LAT_ACCEL', 'Elevation']] = standard_scaler.transform(test_x[['CT','MS','MC','LAT_ACCEL', 'Elevation']])

#### 4. 분석 모델 설계

선형 모델인 Linear Regression, 트리기반 모델 중 배깅 기법을 적용한 Random Forest와 ExtraTrees, 트리기반 모델 중 부스팅 기법을 적용한 XGBoost와 Light GBM을 사용하여 어떤 모델이 좋은 성능을 보여주는지 실험합니다.

In [ ]:
lr = LinearRegression()
rf = RandomForestRegressor()
et = ExtraTreesRegressor()
xgb = XGBRegressor()
lgbm = LGBMRegressor()

#### 4-1. 분석 모델 성능 평가

성능을 평가하기 위해 학습 데이터셋에서 20%를 활용하여 검증 데이터셋을 만듭니다. <br>
이때, 분할된 학습 데이터를 사용하여 최종 모델을 학습시키는 것은 검증 데이터셋으로 할당한 20%만큼의 데이터 손실이 발생한다는 점을 유의하시기 바랍니다.

In [ ]:
train_x_splited, val_x, train_y_splited, val_y = train_test_split(scaled_train_x, train_y, test_size=0.2, random_state=0)

In [ ]:
lr.fit(train_x_splited, train_y_splited)
lr_pred = lr.predict(val_x)
print('Linear Regression 모델의 MAE : ', mean_absolute_error(val_y, lr_pred))

rf.fit(train_x_splited, train_y_splited)
rf_pred = rf.predict(val_x)
print('Random Forest 모델의 MAE : ', mean_absolute_error(val_y, rf_pred))

et.fit(train_x_splited, train_y_splited)
et_pred = et.predict(val_x)
print('Extra Trees 모델의 MAE : ', mean_absolute_error(val_y, et_pred))

xgb.fit(train_x_splited, train_y_splited)
xgb_pred = xgb.predict(val_x)
print('XGBoost 모델의 MAE : ', mean_absolute_error(val_y, xgb_pred))

lgbm.fit(train_x_splited, train_y_splited)
lgbm_pred = lgbm.predict(val_x)
print('LGBM 모델의 MAE : ', mean_absolute_error(val_y, lgbm_pred))

#### 4-2. GridSearchCV(GridSearch Cross Validation)를 활용한 하이퍼파라미터 튜닝

이제 검증 데이터셋을 이용한 평가에서 가장 좋은 성능을 보인 Extra Trees 모델의 하이퍼파라미터 튜닝을 진행하기 위해 GridSearch CV를 활용합니다. <br>
많은 조합을 한 번에 진행할 수 있지만, 시간관계상 random_state 값만 찾도록 하겠습니다.

In [ ]:
param_grid = [
    {'n_estimators' : [100],
     'min_samples_leaf' : [1],
     'random_state' : [0,23,37,42]}
]

grid_search = GridSearchCV(et,
                           param_grid,
                           cv=3,
                           scoring='neg_mean_absolute_error',
                           return_train_score=True,
                           n_jobs=-1)

grid_search.fit(scaled_train_x, train_y)

print('최적의 하이퍼파라미터 : ', grid_search.best_params_)

#### 5. 모델 학습

In [ ]:
et = ExtraTreesRegressor(**grid_search.best_params_) #최적 하이퍼파라미터를 불러와 모델을 구성
et.fit(scaled_train_x, train_y)

#### 6. 예측값 생성

In [ ]:
pred = et.predict(scaled_test_x)

#### 7. 제출 파일 생성

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.head(5)

In [ ]:
submission['HR'] = pred
submission.head(5)

In [ ]:
submission.to_csv('./submission.csv', index = False)